In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [2]:
df_house = pd.read_csv('train.csv')
df_house.drop('Id', axis=1, inplace=True)
df_house

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [3]:
print('Процент незаполненных значений в признаках:\n')
for column_name in df_house.columns:
    col_stat = round(df_house[column_name].isna().mean() * 100, 3)
    print(f'{column_name} - {col_stat}%')

Процент незаполненных значений в признаках:

MSSubClass - 0.0%
MSZoning - 0.0%
LotFrontage - 17.74%
LotArea - 0.0%
Street - 0.0%
Alley - 93.767%
LotShape - 0.0%
LandContour - 0.0%
Utilities - 0.0%
LotConfig - 0.0%
LandSlope - 0.0%
Neighborhood - 0.0%
Condition1 - 0.0%
Condition2 - 0.0%
BldgType - 0.0%
HouseStyle - 0.0%
OverallQual - 0.0%
OverallCond - 0.0%
YearBuilt - 0.0%
YearRemodAdd - 0.0%
RoofStyle - 0.0%
RoofMatl - 0.0%
Exterior1st - 0.0%
Exterior2nd - 0.0%
MasVnrType - 0.548%
MasVnrArea - 0.548%
ExterQual - 0.0%
ExterCond - 0.0%
Foundation - 0.0%
BsmtQual - 2.534%
BsmtCond - 2.534%
BsmtExposure - 2.603%
BsmtFinType1 - 2.534%
BsmtFinSF1 - 0.0%
BsmtFinType2 - 2.603%
BsmtFinSF2 - 0.0%
BsmtUnfSF - 0.0%
TotalBsmtSF - 0.0%
Heating - 0.0%
HeatingQC - 0.0%
CentralAir - 0.0%
Electrical - 0.068%
1stFlrSF - 0.0%
2ndFlrSF - 0.0%
LowQualFinSF - 0.0%
GrLivArea - 0.0%
BsmtFullBath - 0.0%
BsmtHalfBath - 0.0%
FullBath - 0.0%
HalfBath - 0.0%
BedroomAbvGr - 0.0%
KitchenAbvGr - 0.0%
KitchenQual - 0.

1. MSSubClass: Тип жилья для продажи. Категориальное номинальное значение. Требуется One Hot Encoding. Пропусков нет

2. MSZoning: Определяет общую классификацию зонирования продажи. Требуется One Hot Encoding.

3. LotFrontage: Линейные метры улицы, соединенной с недвижимостью. Требуется заполнение пропущенных данных (17.74%). Есть выбросы, оставляем данные с LotFrontage <= 100. Пропуски удалим.

4. LotArea: Размер лота в квадратных футах. Есть выборосы, оставляем данные с LotArea <= 15000

5. Street: Тип подъездной дороги к объекту недвижимости. Категориальное номинальное значение. Требуется One Hot Encoding. 

6. Alley: Тип подъездной аллеи к собственности. Категориальное номинальное значение. Требуется One Hot Encoding. Здесь NaN обозначает отсутствие подъездной аллеи, необходимо его заменить на текстовое значение "None". Добавим новый признак наличия аллеи.

7. LotShape: Общая форма собственности. Категориальное номинальное значение. Требуется One Hot Encoding. 

8. LandContour: Плоскостность участка. Категориальное номинальное значение. Требуется One Hot Encoding.

9. Utilities: Тип доступных коммунальных услуг. Категориальное порядковое значение. Требуется LabelEncoder.

10. LotConfig: Конфигурация лота. Категориальное номинальное значение. Требуется One Hot Encoding.

11. LandSlope: склон собственности. Ксатегориальное порядковое значение. Требуется LabelEncoder

12. Neighborhood: Район. Категориальное номинальное значение. Требуется One Hot Encoding.

13. Condition1: Proximity to various conditions. Категориальное номинальное значение. Требуется One Hot Encoding

14. Condition2: Proximity to various conditions (if more than one is present). Категориальное номинальное значение. Требуется One Hot Encoding и конкатенация с Condition1

15. BldgType: Тип жилого помещения. Категориальное номинальное значение. Требуется One Hot Encoding

16. HouseStyle: Стиль дома. Категориальное номинальное значение. Требуется One Hot Encoding

17. OverallQual (Общее качество): Оценивает общий материал и отделку дома

18. OverallCond (Общее состояние): Оценивает общее состояние дома

19. YearBuilt: Дата строительства дома

20. YearRemodAdd: Дата реконструкции (такая же, как дата строительства, если нет реконструкции или дополнений)

21. RoofStyle: Тип крыши. Категориальное номинальное значение. Требуется One Hot Encoding

22. RoofMatl: Материал крыши. Категориальное номинальное значение. Требуется One Hot Encoding

23. Exterior1st: Наружное покрытие дома. Категориальное номинальное значение. Требуется One Hot Encoding

24. Exterior2nd: Наружное покрытие дома (если более одного материала). Категориальное номинальное значение. Требуется One Hot Encoding и конкатенация с Exterior1st

25. MasVnrType: Тип облицовки каменной кладки. Категориальное номинальное значение. Требуется One Hot Encoding. NaN обозначает отсутствие облицовки, требуется замена на текстовый None. Создадим новый признак наличия облицовки каменной кладки.

26. MasVnrArea: Площадь облицовки каменной кладки в квадратных футах. Пропуски заполняем нулевым значением, так как это означает, что в MasVnrType отсутствует облицовка.

27. ExterQual: Оценивает качество материала на внешней стороне. Категориальное порядковое значение, требуется LabelEncoder

28. ExtraCond: Оценивает текущее состояние материала на внешней стороне. Категориальное порядковое значение, требуется LabelEncoder

29. Foundation: Тип фундамента. Категориальное номинальное значение. Требуется One Hot Encoding.

30. BsmtQual: Оценивает высоту подвала. Категориальное порядковое значение, требуется LabelEncoder. Также NaN требуется заменить на текстовое "None", это означает отсутствие подвала. Создадим новый признак наличия подвала.

31. BsmtCond: Оценивает общее состояние подвала. Категориальное порядковое значение, требуется LabelEncoder. Также NaN требуется заменить на текстовое "None", это означает отсутствие подвала.

32. BsmtExposure: Экспозиция подвала. Категориальное порядковое значение, требуется LabelEncoder. Также NaN требуется заменить на текстовое "None", это означает отсутствие подвала.

33. BsmtFinType1: Оценка готовой площади подвала. Категориальное порядковое значение, требуется LabelEncoder. Также NaN требуется заменить на текстовое "None", это означает отсутствие подвала.

34. BsmtFinSF1: готовые квадратные футы подвала типа 1

35. BsmtFinType2: Оценка готовой площади подвала (если выбрано несколько). Категориальное порядковое значение, требуется LabelEncoder. Также NaN требуется заменить на текстовое "None", это означает отсутствие подвала.

36. BsmtFinSF2:готовые квадратные футы подвала типа 1

37. BsmtUnfSF: незаконченное кв. м площадь подвала

38. TotalBsmtSF: Общая площадь подвала в квадратных футах

39. Heating: Тип отопления. Категориальное номинальное значение. Требуется One Hot Encoding

40. HeatingQC: Качество и состояние отопления. Категориальное порядковое значение, требуется LabelEncoder.

41. CentralAir: Центральное кондиционирование. Флаг, требуется LabelEncoder.

42. Electrical: Электрическая система. Категориальное номинальное значение. Требуется One Hot Encoding. Требуется заполнение пропусков (0.094%). Заполним модой.

43. 1stFlrSF: Первый этаж квадратных футов
 
44. 2ndFlrSF: Второй этаж квадратных футов

45. LowQualFinSF: Низкокачественные готовые квадратные футы (все этажи)

46. GrLivArea: Above grade (ground) living area square feet

47. BsmtFullBath: Basement full bathrooms

48. BsmtHalfBath: Basement half bathrooms

49. FullBath: Full bathrooms above grade

50. HalfBath: Half baths above grade

51. BedroomAbvGr: Bedrooms above grade (does NOT include basement bedrooms)

52. KitchenAbvGr: Kitchens above grade

53. KitchenQual: Качество кухни. Категориальное номинальное значение. Требуется One Hot Encoding. 

54. TotRmsAbvGrd: Общее количество номеров высокого класса (не включает ванные комнаты)

55. Functional: Домашняя функциональность. Категориальное номинальное значение. Требуется One Hot Encoding. 

56. Fireplaces: Количество каминов

57. FireplaceQu: Качество камина. Категориальное номинальное значение. Требуется One Hot Encoding. Требуется заполнение пропусков (49.925%). Отсутствующие значения заполняем текстовым None, это означает отсутствие камина. Создадим новый признак наличия камина.

58. GarageType: Расположение гаража. Категориальное номинальное значение. Требуется One Hot Encoding. Требуется заполнение пропусков (5.904%). Отсутствующие значения заполняем текстовым None, это означает отсутствие гаража. Так же создадим дополнительный признак: 0 - гаража нет, 1 - гараж есть

59. GarageYrBlt: Год постройки гаража. Требуется заполнение пропусков (5.904%). Пропуски стоят тоолько при отсутствии гаража. Заполним их 0.

60. GarageFinish: Внутренняя отделка гаража. Категориальное номинальное значение. Требуется One Hot Encoding. Отсутствующие значения заполняем текстовым None, это означает отсутствие гаража

61. GarageCars: Размер гаража по вместимости автомобиля

62. GarageArea: Размер гаража в квадратных футах

63. GarageQual: Качество гаража. Категориальное порядковое значение, требуется LabelEncoder. Промущенные значение заполняем строкой 'None', это означает отсутствие гаража.

64. GarageCond: Состояние гаража. Категориальное порядковое значение, требуется LabelEncoder. Промущенные значение заполняем строкой 'None', это означает отсутствие гаража.

65. PavedDrive: Асфальтированная подъездная дорожка. Категориальное номинальное значение. Требуется One Hot Encoding.

66. WoodDeckSF: Площадь деревянного настила в квадратных футах

67. OpenPorchSF: Площадь открытой веранды в квадратных футах

68. EnclosedPorch: Площадь крыльца в квадратных футах.

69. 3SsnPorch: Площадь крыльца на три сезона в квадратных футах

70. ScreenPorch: Площадь крыльца экрана в квадратных футах

71. PoolArea: Площадь бассейна в квадратных футах. 

72. PoolQC: качество бассейна. Категориальное порядковое значение, требуется LabelEncoder. Требуется заполнение пропусков (99.521%). Пропущенные значение заполняем строкой 'None', это означает отсутствие бассейна. Так же создадим дополнительный признак: 0 - бассейна нет, 1 - бассейн есть

73. Fence: Качество забора. Категориальное порядковое значение, требуется LabelEncoder. Требуется заполнение пропусков (80.753%). Пропущенные значение заполняем строкой 'None', это означает отсутствие забора. Так же создадим дополнительный признак: 0 - забора нет, 1 - забор есть

74. MiscFeature: особенности, не вошедшие в другие категории. Категориальное номинальное значение. Требуется One Hot Encoding. Требуется заполнение пропусков (96.301%). Отсутствующие значения заполняем текстовым None, это означает отсутствие гаража. Так же создадим дополнительный признак: 0 - нет иных особенностей, 1 - есть иные особенности

75. MiscVal: $Value of miscellaneous feature

76. MoSold: месяц продажи.

77. YrSold: год.продажи. Создадим новый признак с датой продажи, который объединит год и месяц. А признаки с отдельными значениями удалим.

78. SaleType: тип продажи. Категориальное номинальное значение. Требуется One Hot Encoding.

79. SaleCondition: Condition of sale. Категориальное номинальное значение. Требуется One Hot Encoding.

In [4]:
df_house = df_house[df_house.LotFrontage <= 100]
df_house = df_house[df_house.LotArea < 15000]

# Создаем новый признак наличия подъездной аллеи к собственности
df_house.loc[:, 'Alley_isExist'] = df_house.Alley.apply(lambda x: 0 if pd.isna(x) else 1)
df_house.loc[df_house.Alley.isna(), 'Alley'] = 'None'

mapping_Utilities = {'ELO': 1, 'NoSeWa': 2, 'NoSewr': 3, 'AllPub': 4}
df_house.loc[:, 'Utilities'] = df_house.Utilities.map(mapping_Utilities)

mapping_LandSlope = {'Gtl': 1, 'Mod': 2, 'Sev': 3}
df_house.loc[:, 'LandSlope'] = df_house.LandSlope.map(mapping_LandSlope)

# Создаем новый признак наличия облицовки каменной кладки
df_house.loc[:, 'MasVnr_isExist'] = df_house.MasVnrType.apply(lambda x: 0 if pd.isna(x) else 1)
df_house.loc[df_house.MasVnrType.isna(), 'MasVnrType'] = 'None'

df_house.loc[:, 'Condition1+Condition2'] = df_house['Condition1'] + '+' + df_house['Condition2']
df_house.loc[:, 'Exterior1st+Exterior2nd'] = df_house['Exterior1st'] + '+' + df_house['Exterior2nd']
df_house.drop(['Condition1', 'Condition2', 'Exterior1st', 'Exterior2nd'], axis=1, inplace=True)
df_house.loc[df_house.MasVnrArea.isna(), 'MasVnrArea'] = 0

mapping_ExterQual = {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
df_house.ExterQual = df_house.ExterQual.map(mapping_ExterQual)

mapping_ExterCond = {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
df_house.ExterCond = df_house.ExterCond.map(mapping_ExterCond)

# Создаем новый признак наличия подвала
df_house.loc[:, 'Bsmt_isExist'] = df_house.BsmtQual.apply(lambda x: 0 if pd.isna(x) else 1)
mapping_BsmtQual = {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
df_house.loc[df_house.BsmtQual.isna(), 'BsmtQual'] = 'None'
df_house.loc[:, 'BsmtQual'] = df_house.BsmtQual.map(mapping_BsmtQual)

mapping_BsmtCond = {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
df_house.loc[df_house.BsmtCond.isna(), 'BsmtCond'] = 'None'
df_house.loc[:, 'BsmtCond'] = df_house.BsmtCond.map(mapping_BsmtCond)

df_house.loc[df_house.BsmtExposure.isna(), 'BsmtExposure'] = 'None'
mapping_BsmtExposure = {'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4}
df_house.loc[:, 'BsmtExposure'] = df_house.BsmtExposure.map(mapping_BsmtExposure)

df_house.loc[df_house.BsmtFinType1.isna(), 'BsmtFinType1'] = 'None'
mapping_BsmtFinType1 = {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}
df_house.loc[:, 'BsmtFinType1'] = df_house.BsmtFinType1.map(mapping_BsmtFinType1)

df_house.loc[df_house.BsmtFinType2.isna(), 'BsmtFinType2'] = 'None'
mapping_BsmtFinType2 = mapping_BsmtFinType1
df_house.loc[:, 'BsmtFinType2'] = df_house.BsmtFinType2.map(mapping_BsmtFinType2)

mapping_HeatingQC = {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
df_house.loc[:, 'HeatingQC'] = df_house.HeatingQC.map(mapping_HeatingQC)

mapping_CentralAir = {'N': 0, 'Y': 1}
df_house.loc[:, 'CentralAir'] = df_house.CentralAir.map(mapping_CentralAir)

# Заполняем модой, SBrkr - самое частое значение
df_house.loc[df_house.Electrical.isna(), 'Electrical'] = 'SBrkr'

mapping_KitchenQual = {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
df_house.loc[:, 'KitchenQual'] = df_house.KitchenQual.map(mapping_KitchenQual)

# Создаем новый признак наличия гаража
df_house.loc[:, 'Garage_isExist'] = df_house.GarageType.apply(lambda x: 0 if pd.isna(x) else 1)
df_house.loc[df_house.GarageType.isna(), 'GarageType'] = 'None'

df_house.loc[df_house.GarageYrBlt.isna(), 'GarageYrBlt'] = 0

df_house.loc[df_house.GarageFinish.isna(), 'GarageFinish'] = 'None'

df_house.loc[df_house.GarageQual.isna(), 'GarageQual'] = 'None'
mapping_GarageQual = {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
df_house.loc[:, 'GarageQual'] = df_house.GarageQual.map(mapping_GarageQual)

# Создаем новый признак наличия камина
df_house.loc[:, 'FireplaceQu_isExist'] = df_house.FireplaceQu.apply(lambda x: 0 if pd.isna(x) else 1)
df_house.loc[df_house.FireplaceQu.isna(), 'FireplaceQu'] = 'None'
mapping_FireplaceQu = {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
df_house.loc[:, 'FireplaceQu'] = df_house.FireplaceQu.map(mapping_FireplaceQu)

df_house.loc[df_house.GarageCond.isna(), 'GarageCond'] = 'None'
mapping_GarageCond = {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
df_house.loc[:, 'GarageCond'] = df_house.GarageCond.map(mapping_GarageCond)

# Создаем новый признак наличия бассейна
df_house.loc[:, 'Pool_isExist'] = df_house.PoolQC.apply(lambda x: 0 if pd.isna(x) else 1)
df_house.loc[df_house.PoolQC.isna(), 'PoolQC'] = 'None'
mapping_PoolQC = {'None': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
df_house.loc[:, 'PoolQC'] = df_house.PoolQC.map(mapping_PoolQC)

# Создаем новый признак наличия забора
df_house.loc[:, 'Fence_isExist'] = df_house.Fence.apply(lambda x: 0 if pd.isna(x) else 1)
df_house.loc[df_house.Fence.isna(), 'Fence'] = 'None'
mapping_Fence = {'None': 0, 'MnWw': 1, 'GdWo': 2, 'MnPrv': 3, 'GdPrv': 4}
df_house.loc[:, 'Fence'] = df_house.Fence.map(mapping_Fence)

# Создаем новый признак наличия иных особенностей
df_house.loc[:, 'MiscFeature_isExist'] = df_house.MiscFeature.apply(lambda x: 0 if pd.isna(x) else 1)
df_house.loc[df_house.MiscFeature.isna(), 'MiscFeature'] = 'None'

In [5]:
one_hot_features = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'LotConfig',
                   'Neighborhood', 'Condition1+Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 
                   'RoofMatl', 'Exterior1st+Exterior2nd', 'MasVnrType', 'Foundation', 'Heating',
                   'Functional', 'GarageType', 'GarageFinish', 'PavedDrive', 'MiscFeature', 'SaleType',
                   'SaleCondition', 'Electrical']
df_house = pd.get_dummies(df_house, columns=one_hot_features)

Проанализируем на предмет выборосов основные численные признаки

In [6]:
df_house[['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
          'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'LowQualFinSF', 'GrLivArea',
          'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 
          'PoolArea']].describe()

,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea
count,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000
mean,65.663543,8707.999063,95.082474,406.343018,40.008435,572.681350,1019.032802,1120.339269,329.966261,5.247423,5.247423,1455.552952,455.505155,84.582943,42.475164,23.183693,3.208997,13.251172,1.147142
std,17.273167,2829.386345,170.170593,414.780252,145.311934,432.069512,395.852586,345.740456,413.469912,43.929893,43.929893,459.613019,211.890044,113.337677,62.610384,60.586703,29.693373,51.845027,26.529673
min,21.000000,1300.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,57.000000,7200.000000,0.000000,0.000000,0.000000,240.500000,776.000000,864.000000,0.000000,0.000000,0.000000,1117.500000,300.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,66.000000,8993.000000,0.000000,349.000000,0.000000,495.000000,960.000000,1056.000000,0.000000,0.000000,0.000000,1412.000000,472.000000,0.000000,20.000000,0.000000,0.000000,0.000000,0.000000
75%,78.000000,10557.000000,150.500000,670.000000,0.000000,810.500000,1247.500000,1327.500000,705.500000,0.000000,0.000000,1717.500000,576.000000,160.000000,63.000000,0.000000,0.000000,0.000000,0.000000
max,100.000000,14963.000000,1600.000000,2188.000000,1474.000000,2046.000000,3206.000000,2524.000000,1611.000000,515.000000,515.000000,3395.000000,1390.000000,736.000000,547.000000,386.000000,508.000000,480.000000,648.000000


Явные выбросы были устранены. Признаки, которые имеют много нулевых значений, принято решений оставить как есть.

Найдем самые значимые признаки с помощью случайного леса.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_house[df_house.columns.drop('SalePrice')],
                                                    df_house['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=42)
rand_forest = RandomForestRegressor(n_estimators=1000)
rand_forest.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1000)

In [8]:
features_importances = sorted(zip(rand_forest.feature_importances_, X_train.columns), reverse=True)
for val, name in features_importances:
    print(f'{name}: {val}')

OverallQual: 0.6535289578384785
GrLivArea: 0.07875768508703847
BsmtFinSF1: 0.025415439687847506
ExterQual: 0.02281137758972538
TotalBsmtSF: 0.02109602359866852
YearBuilt: 0.018584576173163246
GarageCars: 0.01626903217770852
GarageArea: 0.016231682001080043
BsmtQual: 0.015263954733924438
1stFlrSF: 0.012576746557934107
LotArea: 0.00925913857561885
YearRemodAdd: 0.007798338790001757
LotFrontage: 0.0059753319095211645
GarageYrBlt: 0.005267239138943357
GarageType_Attchd: 0.005195290040652447
2ndFlrSF: 0.005117765137869254
BsmtUnfSF: 0.0043877686071328885
OverallCond: 0.0041790185250555395
WoodDeckSF: 0.004075923461238196
KitchenQual: 0.003935946454142492
OpenPorchSF: 0.0038787200967355146
FireplaceQu: 0.003690104941366778
TotRmsAbvGrd: 0.0030789738031615855
MasVnrArea: 0.002786407164848996
BsmtFinType1: 0.002737240594145092
MoSold: 0.0027238811381462566
FullBath: 0.002086917476660799
MSZoning_RM: 0.0019220181924580928
BsmtExposure: 0.001901026825544406
CentralAir: 0.0018403014517953352
Sale

Попробуем построить модель линейной регрессии на всех полученных признаках

In [9]:
lr = LinearRegression()

scaler = StandardScaler()     
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

cv_result = cross_val_score(lr, X=X_train_std, y=y_train, scoring='r2', cv=10)
print(f'mean r2-score: {cv_result.mean()}\n')
for cv, val in enumerate(cv_result, 1):
    print(f'{cv}-fold r2-score: {val}')

mean r2-score: -6.166590489813555e+25

1-fold r2-score: -6.27856794951858e+24
2-fold r2-score: -1.2183292500650867e+22
3-fold r2-score: -3.5935742804489997e+25
4-fold r2-score: -4.410740434357914e+22
5-fold r2-score: -1.931076724149456e+24
6-fold r2-score: -4.9742285003671584e+26
7-fold r2-score: -7.312272986416438e+25
8-fold r2-score: -5.371137347017252e+19
9-fold r2-score: -6.207150905452648e+19
10-fold r2-score: -1.9116751225904963e+24


Получили отрицательное значение, значит ошибка нашей модели значительно больше ошибок модели методом усреднения.
Проверим соотношение количества признаков и количества обучающих образцов.

In [10]:
print(f'Количество признаков в модели: {len(df_house.columns)}')
print(f'Количество обучающих образцов в модели: {len(df_house)}')

Количество признаков в модели: 277
Количество обучающих образцов в модели: 1067


Количество признаков составляет почти четверть количества образцов. 
Попробуем сжать признаки с помощью PCA. Вначале подберем наилушее значение главных компонент.

In [11]:
pca_result = []
for n_components in range(10, len(df_house.columns)):
    pca = PCA(n_components=n_components, random_state=42)
    pca.fit(X_train_std)
    X_train_std_pca = pca.transform(X_train_std)
    X_test_std_pca = pca.transform(X_test_std)
    cv_result = cross_val_score(lr, X=X_train_std_pca, y=y_train, scoring='r2', cv=10)
    pca_result.append((cv_result.mean(), n_components))
    
print(f'Наилучший результат: mean r2-score {round(max(pca_result)[0], 6)}, ' 
      f'количество компонент {max(pca_result)[1]}')

Наилучший результат: mean r2-score 0.86326, количество компонент 56


Теперь найдем статистику по фолдам для нашего наилучшего количества главных компонент

In [12]:
pca = PCA(n_components=56, random_state=42)
pca.fit(X_train_std)
X_train_std_pca = pca.transform(X_train_std)
X_test_std_pca = pca.transform(X_test_std)

cv_result = cross_val_score(lr, X=X_train_std_pca, y=y_train, scoring='r2', cv=10)
print(f'mean r2-score: {cv_result.mean()}\n')
for cv, val in enumerate(cv_result, 1):
    print(f'{cv}-fold r2-score: {val}')

mean r2-score: 0.8632598996315458

1-fold r2-score: 0.9108647060325944
2-fold r2-score: 0.8968720105501269
3-fold r2-score: 0.8891115745516083
4-fold r2-score: 0.853002755521647
5-fold r2-score: 0.9011793390780989
6-fold r2-score: 0.8754410450673764
7-fold r2-score: 0.8182233860778021
8-fold r2-score: 0.7753176216451823
9-fold r2-score: 0.8524541346313079
10-fold r2-score: 0.8601324231597138


Средний результат значительно улучшился и перестал быть отрицательным. В некоторых разделениях даже удается получить оценку более 0.9.

Теперь попробуем обучиться только на тех признаках, которые считаются важными для леса случайных деревьев.

In [13]:
filtered_features = [name for val, name in features_importances if val > 0.001]

X_train, X_test, y_train, y_test = train_test_split(df_house[filtered_features],
                                                    df_house['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=42)

scaler = StandardScaler()     
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

cv_result = cross_val_score(lr, X=X_train_std, y=y_train, scoring='r2', cv=10)
print(f'mean r2-score: {cv_result.mean()}\n')
for cv, val in enumerate(cv_result, 1):
    print(f'{cv}-fold r2-score: {val}')

mean r2-score: 0.885644052405483

1-fold r2-score: 0.9266697315904395
2-fold r2-score: 0.8873910672384412
3-fold r2-score: 0.9021009480259959
4-fold r2-score: 0.9085217077977701
5-fold r2-score: 0.8937829785034481
6-fold r2-score: 0.9241793577327966
7-fold r2-score: 0.8451019735009213
8-fold r2-score: 0.8220324276006588
9-fold r2-score: 0.8597453242574695
10-fold r2-score: 0.8869150078068887


С помощью удаления ненужных признаков мы смогли немного улучшить качество нашей модели по сравнению со сжатием PCA. Теперь попробуем создать модель на основе стекинга.

In [14]:
stack_estimators = [('svr', SVR()),
                    ('lr', LinearRegression()),
                    ('tree_reg', DecisionTreeRegressor(random_state=42)),
                    ('rf_reg', RandomForestRegressor(n_estimators=100, random_state=42)),
                   ]

stack_meta_estimator = LinearRegression()

stack = StackingRegressor(estimators=stack_estimators, final_estimator=stack_meta_estimator, cv=10)

cv_result = cross_val_score(stack, X=X_train_std, y=y_train, scoring='r2', cv=10)
print(f'mean r2-score: {cv_result.mean()}\n')
for cv, val in enumerate(cv_result, 1):
    print(f'{cv}-fold r2-score: {val}')

mean r2-score: 0.901785728756803

1-fold r2-score: 0.9496110651931521
2-fold r2-score: 0.9304356768535316
3-fold r2-score: 0.9085896210634767
4-fold r2-score: 0.9003235266816371
5-fold r2-score: 0.9179892359258458
6-fold r2-score: 0.9373777074102527
7-fold r2-score: 0.8612563162445824
8-fold r2-score: 0.8167612454298405
9-fold r2-score: 0.8741226104631739
10-fold r2-score: 0.9213902823025362


Удалось улучшить результат по сравнению с обычной линейной регрессией. Теперь убедимся, что модель второго уровня действительно улучшает результаты моделей первого уровня.

In [15]:
stack.fit(X_train_std, y_train)

StackingRegressor(cv=10,
                  estimators=[('svr', SVR()), ('lr', LinearRegression()),
                              ('tree_reg',
                               DecisionTreeRegressor(random_state=42)),
                              ('rf_reg',
                               RandomForestRegressor(random_state=42))],
                  final_estimator=LinearRegression())

In [16]:
for name, estimator in stack.named_estimators_.items():
    cv_result = cross_val_score(estimator, X=X_train_std, y=y_train, scoring='r2', cv=10)
    print(f'{name} mean r2-score: {cv_result.mean()}\n')

svr mean r2-score: -0.09204335980098635

lr mean r2-score: 0.885644052405483

tree_reg mean r2-score: 0.7554666970760503

rf_reg mean r2-score: 0.883022711402903



Итоговый средний результат R2 для стекинга: 0.9017. Этот результат превышает все средние оценки моделей первого уровня. Отсюда делаем заключение, что обобщающая модель второго уровня дает лучшие результаты, чем отдельно взятые модели первого уровня.